## ICQA (Air Quality Catalonian Index)

|ICQA|Code|100 a 50|49 a 0|-1 a -50|-51 a -100|
|---|---|---|---|---|---|
|O3 1h(ug/m3)|14|0-110|111-180|181-240|>241|
|PM10 24h(ug/m3)|10|0-35|36-50|51-75|>76|
|CO 8h(mg/m3)|6|0-5|6-10|11-15|>16|
|SO2 1h(ug/m3)|1|0-200|201-350|351-500|>501|
|NO2 1h(ug/m3)|8|0-90|91-200|201-400|>401|
|Table 1|
------------------------------
|Air Quality|Good|Moderate|Poor|
|---|---|---|---|
|ICQA|>=50|0-49|<0|
------------------------------

Get ICQA Codes from https://opendata-ajuntament.barcelona.cat/data/ca/dataset/contaminants-estacions-mesura-qualitat-aire/resource/c122329d-d26d-469e-bf9e-8efa10e4c127

CSV Format changed on 04/04/2019, so we have to normalize data and make both formats similar

In [331]:
# Define a function to calculate ICQA element quality

def get_icqa(element, value):
    match element:
        case "O3":
            if value > 240:
                return -2
            elif value > 180:
                return -1
            elif value > 110:
                return 0
            else: return 1
        case "PM10":
            if value > 75:
                return -2
            elif value > 50:
                return -1
            elif value > 35:
                return 0
            else: return 1
        case "NO2":
            if value > 400:
                return -2
            elif value > 200:
                return -1
            elif value > 90:
                return 0
            else: return 1

def get_air_quality(o3,pm10,no2):
    total = get_icqa("O3",o3) + get_icqa("PM10",pm10) + get_icqa("NO2",no2)
    if total == 3:
        return "Good"
    elif total > 0:
        return "Moderate"
    else: return "Poor"



In [305]:
import pandas as pd

df = pd.read_csv("../downloads/air_stations_Nov2017.csv")
df.head()

,Station,Longitude,Latitude,Ubication,District Name,Neighborhood Name
0,Barcelona - Ciutadella,2.1874,41.3864,Parc de la Ciutadella,Ciutat Vella,"Sant Pere, Santa Caterina i la Ribera"
1,Barcelona - Eixample,2.1538,41.3853,Av. Roma - c/ Comte Urgell,Eixample,la Nova Esquerra de l'Eixample
2,Barcelona - Gràcia,2.1534,41.3987,Plaça Gal·la Placídia (Via Augusta - Travesser...,Gracia,la Vila de Gracia
3,Barcelona - Palau Reial,2.1151,41.3875,c/ John Maynard Keynes - c/ de Jordi Girona,Les Corts,Pedralbes
4,Barcelona - Poblenou,2.2045,41.4039,Plaça Josep Trueta (Pujades - Lope de Vega),Sant Marti,el Poblenou


In [306]:
df['Station'] = df['Station'].apply(lambda row: row.split(" - ")[1])

In [307]:
# df['Station'].unique()
display(df)

,Station,Longitude,Latitude,Ubication,District Name,Neighborhood Name
0,Ciutadella,2.1874,41.3864,Parc de la Ciutadella,Ciutat Vella,"Sant Pere, Santa Caterina i la Ribera"
1,Eixample,2.1538,41.3853,Av. Roma - c/ Comte Urgell,Eixample,la Nova Esquerra de l'Eixample
2,Gràcia,2.1534,41.3987,Plaça Gal·la Placídia (Via Augusta - Travesser...,Gracia,la Vila de Gracia
3,Palau Reial,2.1151,41.3875,c/ John Maynard Keynes - c/ de Jordi Girona,Les Corts,Pedralbes
4,Poblenou,2.2045,41.4039,Plaça Josep Trueta (Pujades - Lope de Vega),Sant Marti,el Poblenou
5,Sants,2.1331,41.3788,Jardins de Can Mantega (Joan Güell - Violant d...,Sants-Montjuic,Sants
6,Vall Hebron,2.1480,41.4261,Parc de la Vall d’Hebron - c/ Martí Codolar - ...,Horta-Guinardo,la Vall d'Hebron
7,"Vallvidrera, el Tibidabo i les Planes",2.1211,41.4176,"Ctra Observatori Fabra, 27",Sarrià-Sant Gervasi,"Vallvidrera, el Tibidabo i les Planes"


## Stations
Get data from API: https://opendata-ajuntament.barcelona.cat/data/api/action/datastore_search?resource_id=3b2c1f22-2a64-40a7-9154-3d0258d847ed

In [308]:
import requests

response = requests.get("https://opendata-ajuntament.barcelona.cat/data/api/action/datastore_search?resource_id=3b2c1f22-2a64-40a7-9154-3d0258d847ed")
data = response.json()

In [309]:
stations = pd.DataFrame().from_dict(data['result']['records'])
stations.head()

,Latitud,Codi_Contaminant,nom_cabina,Clas_2,Nom_districte,codi_dtes,Codi_districte,Nom_barri,Estacio,Clas_1,codi_eoi,Codi_barri,_id,ubicacio,Longitud,zqa
0,41.3864,8,Barcelona - Ciutadella,Fons,Ciutat Vella,IL,1,"Sant Pere, Santa Caterina i la Ribera",50,Urbana,8019050,4,1,Parc de la Ciutadella,2.1874,1
1,41.3864,14,Barcelona - Ciutadella,Fons,Ciutat Vella,IL,1,"Sant Pere, Santa Caterina i la Ribera",50,Urbana,8019050,4,2,Parc de la Ciutadella,2.1874,1
2,41.3864,7,Barcelona - Ciutadella,Fons,Ciutat Vella,IL,1,"Sant Pere, Santa Caterina i la Ribera",50,Urbana,8019050,4,3,Parc de la Ciutadella,2.1874,1
3,41.3864,12,Barcelona - Ciutadella,Fons,Ciutat Vella,IL,1,"Sant Pere, Santa Caterina i la Ribera",50,Urbana,8019050,4,4,Parc de la Ciutadella,2.1874,1
4,41.3853,8,Barcelona - Eixample,Trànsit,Eixample,IH,5,la Nova Esquerra de l'Eixample,43,Urbana,8019043,9,5,Av. Roma - c/ Comte Urgell,2.1538,1


In [310]:
stations.drop_duplicates('Estacio',inplace=True)
display(stations)

,Latitud,Codi_Contaminant,nom_cabina,Clas_2,Nom_districte,codi_dtes,Codi_districte,Nom_barri,Estacio,Clas_1,codi_eoi,Codi_barri,_id,ubicacio,Longitud,zqa
0,41.3864,8,Barcelona - Ciutadella,Fons,Ciutat Vella,IL,1,"Sant Pere, Santa Caterina i la Ribera",50,Urbana,8019050,4,1,Parc de la Ciutadella,2.1874,1
4,41.3853,8,Barcelona - Eixample,Trànsit,Eixample,IH,5,la Nova Esquerra de l'Eixample,43,Urbana,8019043,9,5,Av. Roma - c/ Comte Urgell,2.1538,1
16,41.3987,8,Barcelona - Gràcia,Trànsit,Gracia,IJ,6,la Vila de Gracia,44,Urbana,8019044,31,17,Plaça Gal·la Placídia (Via Augusta - Travesser...,2.1534,1
23,41.3875,8,Barcelona - Palau Reial,Fons,Les Corts,IZ,4,Pedralbes,57,Urbana,8019057,21,24,c/ John Maynard Keynes - c/ de Jordi Girona,2.1151,1
31,41.4039,8,Barcelona - Poblenou,Fons,Sant Marti,I2,10,el Poblenou,4,Urbana,8019004,68,32,Plaça Josep Trueta (Pujades - Lope de Vega),2.2045,1
35,41.3788,8,Barcelona - Sants,Fons,Sants-Montjuic,ID,3,Sants,42,Urbana,8019042,18,36,Jardins de Can Mantega (Joan Güell - Violant d...,2.1331,1
38,41.4261,8,Barcelona - Vall Hebron,Fons,Horta-Guinardo,IN,7,la Vall d'Hebron,54,Urbana,8019054,41,39,Parc de la Vall d’Hebron. c/ Martí Codolar - c...,2.1480,1


In [311]:
# Keep only info needed
stations['nom_cabina'] = stations['nom_cabina'].apply(lambda x: x.split(" - ")[1])
display(stations)

,Latitud,Codi_Contaminant,nom_cabina,Clas_2,Nom_districte,codi_dtes,Codi_districte,Nom_barri,Estacio,Clas_1,codi_eoi,Codi_barri,_id,ubicacio,Longitud,zqa
0,41.3864,8,Ciutadella,Fons,Ciutat Vella,IL,1,"Sant Pere, Santa Caterina i la Ribera",50,Urbana,8019050,4,1,Parc de la Ciutadella,2.1874,1
4,41.3853,8,Eixample,Trànsit,Eixample,IH,5,la Nova Esquerra de l'Eixample,43,Urbana,8019043,9,5,Av. Roma - c/ Comte Urgell,2.1538,1
16,41.3987,8,Gràcia,Trànsit,Gracia,IJ,6,la Vila de Gracia,44,Urbana,8019044,31,17,Plaça Gal·la Placídia (Via Augusta - Travesser...,2.1534,1
23,41.3875,8,Palau Reial,Fons,Les Corts,IZ,4,Pedralbes,57,Urbana,8019057,21,24,c/ John Maynard Keynes - c/ de Jordi Girona,2.1151,1
31,41.4039,8,Poblenou,Fons,Sant Marti,I2,10,el Poblenou,4,Urbana,8019004,68,32,Plaça Josep Trueta (Pujades - Lope de Vega),2.2045,1
35,41.3788,8,Sants,Fons,Sants-Montjuic,ID,3,Sants,42,Urbana,8019042,18,36,Jardins de Can Mantega (Joan Güell - Violant d...,2.1331,1
38,41.4261,8,Vall Hebron,Fons,Horta-Guinardo,IN,7,la Vall d'Hebron,54,Urbana,8019054,41,39,Parc de la Vall d’Hebron. c/ Martí Codolar - c...,2.1480,1


In [312]:
stations.drop(['Codi_Contaminant','Clas_2','Clas_1','codi_eoi','Codi_barri','zqa','_id','codi_dtes','Codi_districte'], axis=1,inplace=True)
display(stations)

,Latitud,nom_cabina,Nom_districte,Nom_barri,Estacio,ubicacio,Longitud
0,41.3864,Ciutadella,Ciutat Vella,"Sant Pere, Santa Caterina i la Ribera",50,Parc de la Ciutadella,2.1874
4,41.3853,Eixample,Eixample,la Nova Esquerra de l'Eixample,43,Av. Roma - c/ Comte Urgell,2.1538
16,41.3987,Gràcia,Gracia,la Vila de Gracia,44,Plaça Gal·la Placídia (Via Augusta - Travesser...,2.1534
23,41.3875,Palau Reial,Les Corts,Pedralbes,57,c/ John Maynard Keynes - c/ de Jordi Girona,2.1151
31,41.4039,Poblenou,Sant Marti,el Poblenou,4,Plaça Josep Trueta (Pujades - Lope de Vega),2.2045
35,41.3788,Sants,Sants-Montjuic,Sants,42,Jardins de Can Mantega (Joan Güell - Violant d...,2.1331
38,41.4261,Vall Hebron,Horta-Guinardo,la Vall d'Hebron,54,Parc de la Vall d’Hebron. c/ Martí Codolar - c...,2.1480


## ICQA Codes
Get from https://opendata-ajuntament.barcelona.cat/data/ca/dataset/contaminants-estacions-mesura-qualitat-aire/resource/c122329d-d26d-469e-bf9e-8efa10e4c127


In [313]:
resp = requests.get('https://opendata-ajuntament.barcelona.cat/data/api/action/datastore_search?resource_id=c122329d-d26d-469e-bf9e-8efa10e4c127')
data_codes = resp.json()

In [314]:
icqa_codes = pd.DataFrame.from_dict(data_codes['result']['records'])
display(icqa_codes)

,Codi_Contaminant,Desc_Contaminant,_id,Unitats
0,1,SO2,1,µg/m³
1,7,NO,2,µg/m³
2,8,NO2,3,µg/m³
3,9,PM2.5,4,µg/m³
4,12,NOx,5,µg/m³
5,14,O3,6,µg/m³
6,6,CO,7,mg/m³
7,10,PM10,8,µg/m³
8,22,Black Carbon,9,µg/m³
9,999,Biomassa Black Carbon,10,%


In [315]:
icqa_codes.drop('_id',axis=1,inplace=True)
display(icqa_codes)

,Codi_Contaminant,Desc_Contaminant,Unitats
0,1,SO2,µg/m³
1,7,NO,µg/m³
2,8,NO2,µg/m³
3,9,PM2.5,µg/m³
4,12,NOx,µg/m³
5,14,O3,µg/m³
6,6,CO,mg/m³
7,10,PM10,µg/m³
8,22,Black Carbon,µg/m³
9,999,Biomassa Black Carbon,%


## Air quality data
We have to make this csv look similar to the new format

In [316]:
df2 = pd.read_csv("../downloads/air_quality_Nov2017.csv")
df2.head()

,Station,Air Quality,Longitude,Latitude,O3 Hour,O3 Quality,O3 Value,NO2 Hour,NO2 Quality,NO2 Value,PM10 Hour,PM10 Quality,PM10 Value,Generated,Date Time
0,Barcelona - Sants,Good,2.1331,41.3788,NaN,NaN,NaN,0h,Good,84.0,NaN,NaN,NaN,01/11/2018 0:00,1541027104
1,Barcelona - Eixample,Moderate,2.1538,41.3853,0h,Good,1.0,0h,Moderate,113.0,0h,Good,36.0,01/11/2018 0:00,1541027104
2,Barcelona - Gràcia,Good,2.1534,41.3987,0h,Good,10.0,0h,Good,73.0,NaN,NaN,NaN,01/11/2018 0:00,1541027104
3,Barcelona - Ciutadella,Good,2.1874,41.3864,0h,Good,2.0,0h,Good,86.0,NaN,NaN,NaN,01/11/2018 0:00,1541027104
4,Barcelona - Vall Hebron,Good,2.1480,41.4261,0h,Good,7.0,0h,Good,69.0,NaN,NaN,NaN,01/11/2018 0:00,1541027104


In [317]:
df2['Station'] = df2['Station'].apply(lambda row: row.split(" - ")[1])
df2.head()

,Station,Air Quality,Longitude,Latitude,O3 Hour,O3 Quality,O3 Value,NO2 Hour,NO2 Quality,NO2 Value,PM10 Hour,PM10 Quality,PM10 Value,Generated,Date Time
0,Sants,Good,2.1331,41.3788,NaN,NaN,NaN,0h,Good,84.0,NaN,NaN,NaN,01/11/2018 0:00,1541027104
1,Eixample,Moderate,2.1538,41.3853,0h,Good,1.0,0h,Moderate,113.0,0h,Good,36.0,01/11/2018 0:00,1541027104
2,Gràcia,Good,2.1534,41.3987,0h,Good,10.0,0h,Good,73.0,NaN,NaN,NaN,01/11/2018 0:00,1541027104
3,Ciutadella,Good,2.1874,41.3864,0h,Good,2.0,0h,Good,86.0,NaN,NaN,NaN,01/11/2018 0:00,1541027104
4,Vall Hebron,Good,2.1480,41.4261,0h,Good,7.0,0h,Good,69.0,NaN,NaN,NaN,01/11/2018 0:00,1541027104


In [318]:
'''# Convert datetime to utc
from datetime import datetime

def convert_time(timestamp):
    print(timestamp)
    ts = datetime. strptime(timestamp, '%Y-%m-%d %H:%M:%S')
    return int(datetime.timestamp(ts))

df2['Date Time'] = df2['Date Time'].apply(convert_time)
df2.head()'''
#df2.to_dict(orient='index')
# Fix "Hour" columns using data from "Generated", taking only the left part (hours)
df2['Hour'] = df2['Generated'].apply(lambda x: x.split()[1].split(':')[0])
# Split year, month and day, according to the new format
df2['Year'] = df2['Generated'].apply(lambda x: int(x.split()[0].split("/")[2]))
df2['Month'] = df2['Generated'].apply(lambda x: int(x.split()[0].split("/")[1]))
df2['Day'] = df2['Generated'].apply(lambda x: int(x.split()[0].split("/")[0]))
df2.head(10)

,Station,Air Quality,Longitude,Latitude,O3 Hour,O3 Quality,O3 Value,NO2 Hour,NO2 Quality,NO2 Value,PM10 Hour,PM10 Quality,PM10 Value,Generated,Date Time,Hour,Year,Month,Day
0,Sants,Good,2.1331,41.3788,NaN,NaN,NaN,0h,Good,84.0,NaN,NaN,NaN,01/11/2018 0:00,1541027104,0,2018,11,1
1,Eixample,Moderate,2.1538,41.3853,0h,Good,1.0,0h,Moderate,113.0,0h,Good,36.0,01/11/2018 0:00,1541027104,0,2018,11,1
2,Gràcia,Good,2.1534,41.3987,0h,Good,10.0,0h,Good,73.0,NaN,NaN,NaN,01/11/2018 0:00,1541027104,0,2018,11,1
3,Ciutadella,Good,2.1874,41.3864,0h,Good,2.0,0h,Good,86.0,NaN,NaN,NaN,01/11/2018 0:00,1541027104,0,2018,11,1
4,Vall Hebron,Good,2.1480,41.4261,0h,Good,7.0,0h,Good,69.0,NaN,NaN,NaN,01/11/2018 0:00,1541027104,0,2018,11,1
5,Palau Reial,Good,2.1151,41.3875,23h,Good,11.0,23h,Good,57.0,23h,Good,23.0,01/11/2018 0:00,1541027104,0,2018,11,1
6,Poblenou,Good,2.2045,41.4039,NaN,NaN,NaN,23h,Good,86.0,23h,Good,32.0,01/11/2018 0:00,1541027104,0,2018,11,1
7,Observ Fabra,Good,2.1239,41.4183,23h,Good,58.0,23h,Good,3.0,23h,Good,25.0,01/11/2018 0:00,1541027104,0,2018,11,1
8,Sants,Good,2.1331,41.3788,NaN,NaN,NaN,0h,Good,62.0,NaN,NaN,NaN,01/11/2018 1:00,1541030725,1,2018,11,1
9,Eixample,Good,2.1538,41.3853,0h,Good,6.0,0h,Good,80.0,1h,Good,35.0,01/11/2018 1:00,1541030725,1,2018,11,1


In [319]:
# Drop unused columns
df2.drop(['O3 Hour','NO2 Hour','PM10 Hour','Air Quality','O3 Quality','NO2 Quality','PM10 Quality','Generated','Date Time'],axis=1,inplace=True)
df2.rename(columns={'O3 Value':'O3','PM10 Value':'PM10','NO2 Value':'NO2'}, inplace=True)
df2.head()

,Station,Longitude,Latitude,O3,NO2,PM10,Hour,Year,Month,Day
0,Sants,2.1331,41.3788,NaN,84.0,NaN,0,2018,11,1
1,Eixample,2.1538,41.3853,1.0,113.0,36.0,0,2018,11,1
2,Gràcia,2.1534,41.3987,10.0,73.0,NaN,0,2018,11,1
3,Ciutadella,2.1874,41.3864,2.0,86.0,NaN,0,2018,11,1
4,Vall Hebron,2.1480,41.4261,7.0,69.0,NaN,0,2018,11,1


In [320]:
# Count number of nulls
df2.isna().sum()

Station         0
Longitude       0
Latitude        0
O3           1643
NO2           284
PM10         2097
Hour            0
Year            0
Month           0
Day             0
dtype: int64

In [332]:
df2['Air Quality'] = df2.apply(lambda row: get_air_quality(row['O3'],row['PM10'],row['NO2']), axis=1)
df2.head(10)

,Station,Longitude,Latitude,O3,NO2,PM10,Hour,Year,Month,Day,Air Quality
0,Sants,2.1331,41.3788,NaN,84.0,NaN,0,2018,11,1,Good
1,Eixample,2.1538,41.3853,1.0,113.0,36.0,0,2018,11,1,Moderate
2,Gràcia,2.1534,41.3987,10.0,73.0,NaN,0,2018,11,1,Good
3,Ciutadella,2.1874,41.3864,2.0,86.0,NaN,0,2018,11,1,Good
4,Vall Hebron,2.1480,41.4261,7.0,69.0,NaN,0,2018,11,1,Good
5,Palau Reial,2.1151,41.3875,11.0,57.0,23.0,0,2018,11,1,Good
6,Poblenou,2.2045,41.4039,NaN,86.0,32.0,0,2018,11,1,Good
7,Observ Fabra,2.1239,41.4183,58.0,3.0,25.0,0,2018,11,1,Good
8,Sants,2.1331,41.3788,NaN,62.0,NaN,1,2018,11,1,Good
9,Eixample,2.1538,41.3853,6.0,80.0,35.0,1,2018,11,1,Good


In [336]:
df2['District Name'] = df2['Station'].map(df.set_index('Station')['District Name'])
df2['Neighborhood Name'] = df2['Station'].map(df.set_index('Station')['Neighborhood Name'])

df2.head()

,Station,Longitude,Latitude,O3,NO2,PM10,Hour,Year,Month,Day,Air Quality,District Name,Neighborhood Name
0,Sants,2.1331,41.3788,NaN,84.0,NaN,0,2018,11,1,Good,Sants-Montjuic,Sants
1,Eixample,2.1538,41.3853,1.0,113.0,36.0,0,2018,11,1,Moderate,Eixample,la Nova Esquerra de l'Eixample
2,Gràcia,2.1534,41.3987,10.0,73.0,NaN,0,2018,11,1,Good,Gracia,la Vila de Gracia
3,Ciutadella,2.1874,41.3864,2.0,86.0,NaN,0,2018,11,1,Good,Ciutat Vella,"Sant Pere, Santa Caterina i la Ribera"
4,Vall Hebron,2.1480,41.4261,7.0,69.0,NaN,0,2018,11,1,Good,Horta-Guinardo,la Vall d'Hebron


In [337]:
df2.to_dict('records')

[{'Station': 'Sants',
  'Longitude': 2.1331,
  'Latitude': 41.3788,
  'O3': nan,
  'NO2': 84.0,
  'PM10': nan,
  'Hour': '0',
  'Year': 2018,
  'Month': 11,
  'Day': 1,
  'Air Quality': 'Good',
  'District Name': 'Sants-Montjuic',
  'Neighborhood Name': 'Sants'},
 {'Station': 'Eixample',
  'Longitude': 2.1538,
  'Latitude': 41.3853,
  'O3': 1.0,
  'NO2': 113.0,
  'PM10': 36.0,
  'Hour': '0',
  'Year': 2018,
  'Month': 11,
  'Day': 1,
  'Air Quality': 'Moderate',
  'District Name': 'Eixample',
  'Neighborhood Name': "la Nova Esquerra de l'Eixample"},
 {'Station': 'Gràcia',
  'Longitude': 2.1534,
  'Latitude': 41.3987,
  'O3': 10.0,
  'NO2': 73.0,
  'PM10': nan,
  'Hour': '0',
  'Year': 2018,
  'Month': 11,
  'Day': 1,
  'Air Quality': 'Good',
  'District Name': 'Gracia',
  'Neighborhood Name': 'la Vila de Gracia'},
 {'Station': 'Ciutadella',
  'Longitude': 2.1874,
  'Latitude': 41.3864,
  'O3': 2.0,
  'NO2': 86.0,
  'PM10': nan,
  'Hour': '0',
  'Year': 2018,
  'Month': 11,
  'Day': 1,


## Subir los datos a MongoDB

In [342]:
from pymongo import MongoClient
from dotenv import load_dotenv
import os

load_dotenv()

db = MongoClient(os.getenv("URL")).core

In [344]:
db.pollution.insert_many(df2.to_dict('records'))

In [345]:
db['pollution'].distinct("Station")

['Ciutadella',
 'Eixample',
 'Gràcia',
 'Observ Fabra',
 'Palau Reial',
 'Poblenou',
 'Sants',
 'Vall Hebron']